In [1]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 2.9 MB/s et

In [2]:
! pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.2 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
baseline_model_path = "facebook/opt-1.3b"
finetuned_model_path = "/content/drive/MyDrive/finetune_1.3b"
unlearned_model_path = "/content/drive/MyDrive/opt1.3b_unlearned_tested"

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
import re
from wordcloud import WordCloud, STOPWORDS
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
import logging

import os
import zipfile
import warnings
from peft import AdaLoraConfig, TaskType, get_peft_model, PeftModel
from transformers import AutoTokenizer, pipeline
import torch
import argparse
import logging
import random
import time

import numpy as np
import torch
from torch.optim import AdamW
from transformers import AutoModelForCausalLM, AutoTokenizer, get_scheduler

device = torch.device('cuda')

In [6]:
model = AutoModelForCausalLM.from_pretrained(baseline_model_path, return_dict=True)
tokenizer = AutoTokenizer.from_pretrained(baseline_model_path)

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [7]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [8]:
model.to(device)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
          (final_layer_norm): LayerN

In [9]:

def generate_1(prompt: str, max_new_tokens: int, temperature: float, do_sample: bool,model,tokenizer) -> None:

        # Load the Lora model
        model = PeftModel.from_pretrained(model, finetuned_model_path)

        # Generate text
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        tokens = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=do_sample,
            top_p=0.95, top_k=50,
        )
        print(tokenizer.decode(tokens[0]))
        return tokenizer.decode(tokens[0])

In [10]:
def predict_1(prompt):
    completion = generate_1(prompt, 50, 0.3, True,model,tokenizer)
    return completion

In [11]:
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
def generate_2(prompt: str, max_new_tokens: int, temperature: float, do_sample: bool,model,tokenizer) -> None:
        # Import the model
        # Load the Lora model
        model = PeftModel.from_pretrained(model, unlearned_model_path)

        # Generate text
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        tokens = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=do_sample,
            top_p=0.95, top_k=50,
        )
        print(tokenizer.decode(tokens[0]))
        return tokenizer.decode(tokens[0])

In [12]:
def predict_2(prompt):
    completion = generate_2(prompt, 256, 0.3, True,model,tokenizer)
    return completion

In [13]:
import numpy as np
import gradio as gr

demo = gr.Blocks()

with demo:
    with gr.Row():
        gr.HTML("<center><h1> MACHINE UNLEARNING IN LARGE LANGUAGE MODELS </h1></center>")
    with gr.Row():


        gr.HTML('<center><img src="https://swisscognitive.ch/wp-content/uploads/2021/11/ML_in_Cybersecurity_SwissCognitive_World_Leading_AI_Network_3.jpg" width=600px></center>')



    with gr.Tabs():
        with gr.TabItem("Original Finetuned model - Opt-1.3b (Before Unlearning)"):
            with gr.Row():
                demo_1 = gr.Interface(fn=predict_1,
                                      inputs="text", outputs="text", flagging_options=[])

        with gr.TabItem("Unlearned model - Opt-1.3b (After Unlearning)"):
            with gr.Row():
                demo_2 = gr.Interface(fn=predict_2,
                                      inputs="text", outputs="text", flagging_options=[])

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7e7737487bd495f0fc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
